# Explorations about the LDL2022 paper with Ch. Chiarcos

Purpose of this notebook is to try to evaluate some results about the LDL 2022 paper from Chiarcos and Sérasset.


In [32]:
%lsmagic

Available magics:
%auth  %display  %endpoint  %format  %graph  %header  %http_header  %lang  %load  %log  %lsmagics  %method  %outfile  %prefix  %qparam  %show

%auth (basic|digest|none) <username> <passwd> : send HTTP authentication (use env:<var> to get values from environment variables)
%display raw | table [withtypes] | diagram [svg|png] [withliterals] : set display format
%endpoint <url> : set SPARQL endpoint. **REQUIRED**
%format JSON | N3 | XML | default | any | none : set requested result format
%graph <uri> : set default graph for the queries
%header <string> | OFF : add a persistent SPARQL header line before all queries, or delete all defined headers
%http_header <name> [<value>] : add (or delete) an arbitrary HTTP header to all queries
%lang <lang> [...] | default | all : language(s) preferred for labels
%load <filename> : load a file with magic lines and process them
%log critical | error | warning | info | debug : set logging level
%lsmagics  : list all magics
%method get | post : set HTTP method
%outfile <filename> | off : save raw output to a file (use "%d" in name to add cell number, "off" to cancel saving)
%prefix <name> [<uri>] : set (or delete) a persistent URI prefix for all queries
%qparam <name> [<value>] : add (or delete) a persistent custom parameter to all queries
%show <n> | all : maximum number of shown results

In [9]:
%endpoint http://dbpedia.org/sparql

  SELECT distinct *
  WHERE {
    ?a rdfs:label "Pferd"@de.
  } LIMIT 10


Endpoint set to: http://dbpedia.org/sparql

a
http://dbpedia.org/ontology/Horse


In [21]:
SELECT distinct *
WHERE {
  ?a rdfs:label "Horse"@en.
  ?a rdf:type ?type.
  FILTER(!strstarts(str(?type),
  'http://www.w3.org'))
  ?type rdfs:label ?category.
  FILTER(lang(?category)="de")
} LIMIT 10

a,type,category
http://dbpedia.org/resource/Horse,http://dbpedia.org/ontology/Animal,Tier
http://dbpedia.org/resource/Horse,http://dbpedia.org/ontology/PersonFunction,Funktion einer Person


In [23]:
SELECT distinct *
WHERE {
  ?a rdfs:label "horse"@en.
  ?a rdf:type ?type.
  FILTER(!strstarts(str(?type),
  'http://www.w3.org'))
} LIMIT 10

a,type


### Q1: how much categories from DBpedia have a German label ?

First, count the number of classes, then, same thing with w3c classes removed:

In [48]:
%display table
SELECT COUNT distinct *
WHERE {
    ?class a owl:Class.
}


Display: table

callret-0
1189


In [49]:
%display table
SELECT COUNT distinct ?class
WHERE {
    ?class a owl:Class.
    FILTER(!strstarts(str(?class),
              'http://www.w3.org'))
}


Display: table

callret-0
1187


In [50]:
%display table
SELECT distinct *
WHERE {
    ?class a owl:Class.
    FILTER(!strstarts(str(?class),
              'http://www.w3.org'))
} LIMIT 100


Display: table

class
http://dbpedia.org/ontology/Company
http://dbpedia.org/ontology/Activity
http://dbpedia.org/ontology/Name
http://dbpedia.org/ontology/Person
http://dbpedia.org/ontology/Actor
http://dbpedia.org/ontology/Place
http://dbpedia.org/ontology/Publisher
http://dbpedia.org/ontology/Genre
http://dbpedia.org/ontology/Language
http://dbpedia.org/ontology/Department


Then check how many different languages are available in category labels:

In [51]:
SELECT (COUNT distinct ?cl as ?cln), (COUNT distinct ?class as ?classn)
WHERE {
    ?class rdfs:label ?cl;
        a owl:Class.
    FILTER(lang(?cl)="de")
}

Error: SPARQL error: QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 1: syntax error at \'distinct\' before \'?cl\'\n\nSPARQL query:\nSELECT (COUNT distinct ?cl as ?cln), (COUNT distinct ?class as ?classn)\nWHERE {\n    ?class rdfs:label ?cl;\n        a owl:Class.\n    FILTER(lang(?cl)="de")\n}'

In [41]:
%display raw
SELECT distinct ?lang
WHERE {
    ?class rdfs:label ?cl;
        a owl:Class.
    BIND(lang(?cl) as ?lang)
} LIMIT 1000

Display: raw

<sparql xmlns="http://www.w3.org/2005/sparql-results#" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/sw/DataAccess/rf1/result2.xsd">
 <head>
  <variable name="lang"/>
 </head>
 <results distinct="false" ordered="true">
  <result>
   <binding name="lang"><literal></literal></binding>
  </result>
  <result>
   <binding name="lang"><literal>en</literal></binding>
  </result>
  <result>
   <binding name="lang"><literal>de</literal></binding>
  </result>
  <result>
   <binding name="lang"><literal>ja</literal></binding>
  </result>
  <result>
   <binding name="lang"><literal>ko</literal></binding>
  </result>
  <result>
   <binding name="lang"><literal>nl</literal></binding>
  </result>
  <result>
   <binding name="lang"><literal>ga</literal></binding>
  </result>
  <result>
   <binding name="lang"><literal>es</literal></binding>
  </result>
  <result>
   <binding name="lang"><literal>pt</literal></binding>
  </result>
  <result>
   <bindin

In [42]:
%show 50
%display table withtypes
SELECT distinct *
WHERE {
    ?class rdfs:label ?cl;
        a owl:Class.
    FILTER(lang(?cl)="de")
} LIMIT 1000


Result maximum size: 50

<sparql xmlns="http://www.w3.org/2005/sparql-results#" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/sw/DataAccess/rf1/result2.xsd">
 <head>
  <variable name="class"/>
  <variable name="cl"/>
 </head>
 <results distinct="false" ordered="true">
  <result>
   <binding name="class"><uri>http://dbpedia.org/ontology/Company</uri></binding>
   <binding name="cl"><literal xml:lang="de">Unternehmen</literal></binding>
  </result>
  <result>
   <binding name="class"><uri>http://dbpedia.org/ontology/Activity</uri></binding>
   <binding name="cl"><literal xml:lang="de">Aktivität</literal></binding>
  </result>
  <result>
   <binding name="class"><uri>http://dbpedia.org/ontology/Name</uri></binding>
   <binding name="cl"><literal xml:lang="de">Name</literal></binding>
  </result>
  <result>
   <binding name="class"><uri>http://dbpedia.org/ontology/Person</uri></binding>
   <binding name="cl"><literal xml:lang="de">Person</literal></binding>
  </re